# Project description
SQL Project – HR Analytics (Databricks, Spark SQL)
## Goal
Analyze employee attrition and performance trends
## Data Set
IBM HR Analytics Employee Attrition dataset (Kaggle)
## Skills & Tools shown
Databricks Spark SQL | SQL queries (CTEs, WINDOW FUNCTIONs, etc.) | Data exploration for HR insights
## Some insights
Sales department shows the highest attrition. Overtime is linked to turnover. An attrition risk in younger and mid-career groups


### 1. Set up Dataset

In [0]:
# Read IBM_HR csv file into a Spark DataFrame
df= spark.read.csv('dbfs:/FileStore/IBM_HR.csv', header= True)

In [0]:
# Display the dataframe, view first few rows
display(df.limit(10))

Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,4,Male,79,3,1,Laboratory Technician,4,Single,3068,11864,0,Y,No,13,3,3,80,0,8,2,2,7,7,3,6
59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,3,Female,81,4,1,Laboratory Technician,1,Married,2670,9964,4,Y,Yes,20,4,1,80,3,12,3,2,1,0,0,0
30,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,4,Male,67,3,1,Laboratory Technician,3,Divorced,2693,13335,1,Y,No,22,4,2,80,1,1,2,3,1,0,0,0
38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,4,Male,44,2,3,Manufacturing Director,3,Single,9526,8787,0,Y,No,21,4,2,80,0,10,2,3,9,7,1,8
36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,3,Male,94,3,2,Healthcare Representative,3,Married,5237,16577,6,Y,No,13,3,2,80,2,17,3,2,7,7,7,7


In [0]:
# Register csv file as a SQL temporary view
df.createOrReplaceTempView("ibm_employees")

### 2. Basic SQL queries

In [0]:
%sql
--Count employees by department
select department,count(*) as EmployeeCount
from ibm_employees
group by department
order by EmployeeCount desc;



department,EmployeeCount
Research & Development,961
Sales,446
Human Resources,63


In [0]:
%sql
--Attrition rate by department
select Department,
       sum(case when Attrition = 'Yes' then 1 else 0 end)*1.0/count(*) as AttritionRate
from ibm_employees
group by Department;

Department,AttritionRate
Sales,0.2062780269058296
Research & Development,0.1383975026014568
Human Resources,0.1904761904761905


In [0]:
%sql
--Attrition rate by department, rounded at 2 decimal places
select Department,
       round(
        sum(case when Attrition = 'Yes' then 1 else 0 end) * 100.0 / count(*),2) as AttritionRate
from ibm_employees
group by Department;

Department,AttritionRate
Sales,20.63
Research & Development,13.84
Human Resources,19.05


In [0]:
%sql
--Attrition vs Overtime
select OverTime, count(*) as AttritionCount
from ibm_employees
where Attrition = 'Yes'
group by OverTime;

OverTime,AttritionCount
No,110
Yes,127


In [0]:
%sql
--Attrition by Age Group
select 
    case 
        when Age < 30 then 'Under 30'
        when Age between 30 and 40 then '30-40'
        else 'Over 40'
    end as AgeGroup,
    count(*) as AttritionCount
from ibm_employees
where Attrition = 'Yes'
group by AgeGroup
order by AttritionCount DESC;


AgeGroup,AttritionCount
30-40,94
Under 30,91
Over 40,52


In [0]:
%sql
--Compare the monthly income of employees with attrition vs. without
select JobRole,
       avg(case when Attrition = 'Yes' then MonthlyIncome end) as AvgIncomeLeft,
       avg(case when Attrition = 'No' then MonthlyIncome end) as AvgIncomeStayed
from ibm_employees
group by JobRole;


JobRole,AvgIncomeLeft,AvgIncomeStayed
Sales Executive,7489.0,6804.617100371747
Manufacturing Director,7365.5,7289.925925925926
Laboratory Technician,2919.2580645161293,3337.223350253807
Sales Representative,2364.7272727272725,2798.44
Healthcare Representative,8548.222222222223,7453.55737704918
Research Scientist,2780.468085106383,3328.122448979592
Manager,16797.4,17201.484536082473
Research Director,19395.5,15947.346153846154
Human Resources,3715.75,4391.75


### 3. Advanced SQL queries

In [0]:
%sql
--Subquery example: find employees with above-average performance rating
select EmployeeNumber, JobRole, PerformanceRating
from ibm_employees
where PerformanceRating > (
  select avg(PerformanceRating)
  from ibm_employees
)
limit (20);

EmployeeNumber,JobRole,PerformanceRating
2,Research Scientist,4
10,Laboratory Technician,4
11,Laboratory Technician,4
12,Manufacturing Director,4
27,Sales Representative,4
33,Research Scientist,4
57,Laboratory Technician,4
60,Sales Executive,4
61,Research Scientist,4
62,Sales Executive,4


In [0]:
%sql
--Rank employees by income within department (Window Function):
select EmployeeNumber,
       JobRole,
       Department,
       MonthlyIncome
       , rank() over (partition by Department order by MonthlyIncome desc) as IncomeRank
from ibm_employees;


EmployeeNumber,JobRole,Department,MonthlyIncome,IncomeRank
590,Human Resources,Human Resources,9950,1
1744,Human Resources,Human Resources,9756,2
2040,Human Resources,Human Resources,8837,3
1419,Human Resources,Human Resources,7988,4
1722,Human Resources,Human Resources,6430,5
424,Human Resources,Human Resources,6410,6
760,Human Resources,Human Resources,6389,7
184,Human Resources,Human Resources,6347,8
1231,Human Resources,Human Resources,6272,9
1563,Human Resources,Human Resources,6077,10


In [0]:
%sql
--CTE example: find departments with attrition rate above company average
with attrition_rates as (
  select Department,
         sum(case when Attrition = 'Yes' then 1 else 0 end)*100.0/count(*) as DeptAttritionRate
  from ibm_employees
  group by Department)
  select Department,
         round(DeptAttritionRate,2) as DeptAttritionRate
  from attrition_rates
  where DeptAttritionRate > (select avg(DeptAttritionRate) from attrition_rates);

Department,DeptAttritionRate
Sales,20.63
Human Resources,19.05
